# Classification and Regression

In [1]:
import math
import time
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import edward2 as ed
from dbnn.dbnn import DBNN
from dbnn.och import OCH

In [2]:
savefig = False

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

if not savefig:
    plt.rcParams["figure.figsize"] = (4, 4)
    plt.rcParams["font.size"] = 15
    plt.rcParams["figure.titlesize"] = 25
    plt.rcParams["axes.labelsize"] = 20
    plt.rcParams["xtick.labelsize"] = 15
    plt.rcParams["ytick.labelsize"] = 15
    plt.rcParams["legend.fontsize"] = 13
    plt.rcParams["lines.linewidth"] = 2
else:
    plt.rcParams["figure.figsize"] = (8, 8)
    plt.rcParams["font.size"] = 30
    plt.rcParams["axes.labelsize"] = 53
    plt.rcParams["xtick.labelsize"] = 40
    plt.rcParams["ytick.labelsize"] = 40
    plt.rcParams["legend.fontsize"] = 28
    plt.rcParams["lines.linewidth"] = 4

Constants and hyperparameters:

In [3]:
# paths
dataset_root = 'datasets/'
dnn_path = 'models_checkpoints/dnn'
bnn_path = 'models_checkpoints/bnn'

# experiments
seed = 30
tf.random.set_seed(seed)
np.random.seed(seed)

# och parameters
och_x1_params = {'k': 5, 'l': 5.0, 's': 1.0}
och_x_params = {'k': 5, 'l': 1.0, 's': 1.0}
och_y_params = {'k': 5, 'l': 1.0, 's': 1.0}

# style
alpha = 0.12
colors = ["tab:blue", "tab:green", "tab:purple", "tab:red"]
labels = ["DNN", "MU", "DU", "DBNN"]
guide_linestyle=(0, (1, 1))
linestyles = [(0, (5, 1)), 'solid', (0, (5, 1)), 'solid']

In [4]:
def bnn_normal(model, xs, regularizer=None): 
    logits = [tf.expand_dims(logit, axis=-1) for logit in tf.unstack(model(xs), axis=-1)]
    if regularizer is not None:
        logits[1] = tf.math.maximum(logits[1], math.log(math.exp(regularizer) - 1.))
    return tfd.Normal(loc=logits[0], scale=tf.math.softplus(logits[1]))

def bnn_categorical(model, xs):
    logits = model(xs)
    return tfd.Categorical(logits=logits)

In [5]:
def train_epoch(model, x_train, y_train, batch, optimizer, loss_ftn, *loss_metrics):
    indexes_batch = np.array_split(np.random.permutation(len(x_train)), len(x_train) / batch + 1)
    indexes_batch = [indexes for indexes in indexes_batch if len(indexes) > 0]

    for indexes in indexes_batch:
        xs = tf.stack([x_train[index] for index in indexes])
        ys = tf.stack([y_train[index] for index in indexes])
        train_step(model, xs, ys, optimizer, loss_ftn, *loss_metrics)
    
def train_step(model, x_batch, y_batch, optimizer, loss_ftn, *loss_metrics):
    with tf.GradientTape() as tape:
        losses = loss_ftn(model, x_batch, y_batch)
    gradients = tape.gradient(losses[0], model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    for loss, loss_metric in zip(losses, loss_metrics):
        loss_metric(loss)
        
def dnn_loss_ftn(model, xs, ys, loss_obj):
    return loss_obj(ys, model(xs)),

def bnn_normal_loss_ftn(model, xs, ys, length, regularizer=None):
    nll = - tf.reduce_mean(bnn_normal(model, xs, regularizer).log_prob(ys))
    kl = sum(model.losses) / length
    loss = nll + kl
    return loss, nll

def bnn_categorical_loss_ftn(model, xs, ys, length):
    ys = tf.squeeze(ys, axis=-1)
    nll = - tf.reduce_mean(bnn_cateborical(xs).log_prob(ys))
    kl = sum(model.losses) / length
    loss = nll + kl
    return loss, nll

Table of Contents
* [Toy Example of Regression](#Toy-Example-of-Regression)
* [Occupancy Detection Data Set (Classification)](#Occupancy-Detection-Data-Set-(Classification))
* [Air Quality Data Set (Regression)](#Air-Quality-Data-Set-(Regression))

## Toy Example of Regression

In [ ]:
TOY_REGRESSION_FLAG = "toy-regression"

In [ ]:
def create_dnn():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])

def create_bnn():
    return tf.keras.Sequential([
        tfp.layers.DenseFlipout(10, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(2)
    ])

Generate datasets:

In [ ]:
num = 50
x_train = np.linspace(-3, 3, num=num)
y_train = np.cos(x_train) + np.random.normal(0, 0.1, size=num)
x_train = x_train.astype(np.float32).reshape((num, 1))
y_train = y_train.astype(np.float32).reshape((num, 1))

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_xlim(-5.2, 5.2)
ax.set_ylim(-3.2, 3.2)
ax.scatter(x_train, y_train, color="black", s=30, marker="+", label="Training")

### Train Models

Train DNN:

In [ ]:
epochs, batch = 150, 10

dnn = create_dnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_obj = tf.keras.losses.MSE
loss_ftn = lambda model, xs, ys: dnn_loss_ftn(model, xs, ys, loss_obj)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for epoch in range(epochs):
    train_epoch(dnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss)
    
    if (epoch + 1) % 30 == 0:
        template = 'Epoch {}, Loss: {}'
        print(template.format(epoch + 1,
                              train_loss.result()))
        train_loss.reset_states()
        
dnn.save_weights("%s-%s" % (dnn_path, TOY_REGRESSION_FLAG), save_format='tf')

Train BNN:

In [ ]:
epochs, batch = 300, 10

bnn = create_bnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_ftn = lambda model, xs, ys: bnn_normal_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
        
    if (epoch + 1) % 30 == 0:
        template = 'Epoch {}, Loss: {}, NLL: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (bnn_path, TOY_REGRESSION_FLAG), save_format='tf')

### Test Models

In [ ]:
x_pred = np.linspace(-5, 5, num=50)
x_pred = x_pred.astype(np.float32).reshape((50, 1))

DNN:

In [ ]:
dnn = create_dnn()
dnn.load_weights("%s-%s" % (dnn_path, TOY_REGRESSION_FLAG))

dnn_x_pred, dnn_y_pred, dnn_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    dnn_x_pred.append(x)
    dnn_y_pred.append(dnn(tf.stack([x]))[0].numpy())
    time2 = time.time()
    dnn_runtimes.append(time2 - time1)
    
print("DNN Runtime: %.3f ± %.3f (ms)" % (np.mean(dnn_runtimes) * 10 ** 3, np.std(dnn_runtimes) * 10 ** 3))

MU:

In [ ]:
sample_no = 5
bnn = create_bnn()
bnn.load_weights("%s-%s" % (bnn_path, TOY_REGRESSION_FLAG))

mu_x_pred, mu_y_pred, mu_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    for _ in range(sample_no):
        mu_x_pred.append(x)
        mu_y_pred.append(bnn_normal(bnn, tf.stack([x])).sample()[0].numpy())
    time2 = time.time()
    mu_runtimes.append(time2 - time1)
    
print("MU Runtime: %.3f ± %.3f (ms)" % (np.mean(mu_runtimes) * 10 ** 3, np.std(mu_runtimes) * 10 ** 3))

DU:

In [ ]:
sample_no = 5

x_dims, y_dims = [1], [1]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
du = DBNN(lambda x: dnn(tf.stack([x[0]])), och_x_1, och_x, och_y)

du_x_pred, du_y_pred, du_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    du.update(x)
    time2 = time.time()
    du_runtimes.append(time2 - time1)
    for _ in range(sample_no):
        _x_pred, _y_pred = du.och_x.sample(), du.och_y.sample()
        if _x_pred is not None and _y_pred is not None:
            du_x_pred.append(_x_pred[0])
            du_y_pred.append(_y_pred.numpy())

print("DU Runtime: %.3f ± %.3f (ms)" % (np.mean(du_runtimes) * 10 ** 3, np.std(du_runtimes) * 10 ** 3))

DBNN:

In [ ]:
sample_no = 5

x_dims, y_dims = [1], [1]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
dbnn = DBNN(lambda x: bnn_normal(bnn, tf.stack([x[0]])).sample(), och_x_1, och_x, och_y)

dbnn_x_pred, dbnn_y_pred, dbnn_runtimes = [], [], []
for x in x_pred:
    time1 = time.time()
    dbnn.update(x)
    time2 = time.time()
    dbnn_runtimes.append(time2 - time1)
    for _ in range(sample_no):
        _x_pred, _y_pred = dbnn.och_x.sample(), dbnn.och_y.sample()
        if _x_pred is not None and _y_pred is not None:
            dbnn_x_pred.append(_x_pred[0])
            dbnn_y_pred.append(_y_pred.numpy())

print("DBNN Runtime: %.3f ± %.3f (ms)" % (np.mean(dbnn_runtimes) * 10 ** 3, np.std(dbnn_runtimes) * 10 ** 3))

Visualize results:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 4))
datasets = [(dnn_x_pred, dnn_y_pred), (mu_x_pred, mu_y_pred), (du_x_pred, du_y_pred), (dbnn_x_pred, dbnn_y_pred)]
for ax, preds, label, color in zip(axes, datasets, ["DNN", "MU", "DU", "DBNN"], colors):
    ax.set_xlim(-5.2, 5.2)
    ax.set_ylim(-3.2, 3.2)
    ax.scatter(x_train, y_train, color="black", s=30, marker="+", label="Training")
    ax.scatter(preds[0], preds[1], facecolors="none", edgecolors=color, label=label)
    ax.legend(edgecolor='black', fancybox=False)

## [Occupancy Detection Data Set](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+) (Classification)

In [ ]:
OCC_FLAG = "occupancy"
OCC_INPUT_DIM = 5
OCC_CLASS_NO = 2

In [ ]:
train_file_names, test_file_names = ["datatraining.txt"], ["datatest.txt", "datatest2.txt"]

def occupancy_data(file_names):
    occupancy_path = dataset_root + "%s/" % OCC_FLAG
    x, y = [], []
    for file_name in file_names:
        with open(occupancy_path + file_name, 'r') as csvfile:
            file_reader = csv.reader(csvfile, delimiter=',')
            header = next(file_reader)
            for row in file_reader:
                x.append(tf.constant([float(v) for v in row[2: 7]]))
                y.append(tf.constant([int(row[7])]))
    return x, y

In [ ]:
def create_dnn():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(50, activation=tf.nn.relu),
        tf.keras.layers.Dense(50, activation=tf.nn.relu),
        tf.keras.layers.Dense(OCC_CLASS_NO, activation=tf.nn.sigmoid)
    ])

def create_bnn():
    return tf.keras.Sequential([
        tfp.layers.DenseFlipout(50, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(50, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(OCC_CLASS_NO)
    ])

### Train Models

In [ ]:
x_train, y_train = occupancy_data(train_file_names)

print("train dataset: %d, test datasets: %d" % (len(x_train), len(x_test)))
print("x sample: ", x_train[0].numpy())
print("y sample: ", y_train[0].numpy())

Train DNN:

In [ ]:
epochs, batch = 10, 10

dnn = create_dnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()
loss_ftn = lambda model, xs, ys: dnn_loss_ftn(model, xs, ys, loss_obj)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for epoch in range(epochs):
    train_epoch(dnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss)
    
    if (epoch + 1) % 1 == 0:
        template = 'Epoch {}, Loss: {}'
        print(template.format(epoch + 1,
                              train_loss.result()))
        train_loss.reset_states()
        
dnn.save_weights("%s-%s" % (dnn_path, OCC_FLAG), save_format='tf')

Train BNN:

In [ ]:
epochs, batch = 20, 10

bnn = create_bnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_ftn = lambda model, xs, ys: bnn_categorical_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
        
    if (epoch + 1) % 1 == 0:
        template = 'Epoch {}, Loss: {}, NLL: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (bnn_path, OCC_FLAG), save_format='tf')

### Test Models

In [ ]:
x_test, y_test = occupancy_data(test_file_names)

print("Testset size: ", len(x_test))

In [50]:
def test_step(model, xs, ys, runtime_metric, *loss_metrics):
    for loss_ftn, metric in loss_metrics:
        time1 = time.time()
        for loss_ftn, metric in loss_metrics:
            loss = loss_ftn(model, xs, ys)
            if loss is not None:
                metric(loss)
        time2 = time.time()
        runtime_metric(time2 - time1)
        
def test(model, x_test, y_test, batch, runtime_metric, *loss_metrics):
    indexes_batch = np.array_split(range(len(x_test)), len(x_test) / batch + 1)
    indexes_batch = [indexes for indexes in indexes_batch if len(indexes) > 0]
    for indexes in indexes_batch:
        xs = tf.stack([x_test[index] for index in indexes])
        ys = tf.stack([y_test[index] for index in indexes])
        test_step(model, xs, ys, runtime_metric, *loss_metrics)
        
def msec_from_samples(ys, classes_no, sws):
    return sum([tf.keras.losses.MSE(tf.one_hot(ys, classes_no), s) * w for s, w in sws])

def mse_from_samples(ys, sws):
    return sum([tf.keras.losses.MSE(ys, s) * w for s, w in sws])

cce_object = tf.keras.losses.SparseCategoricalCrossentropy()

def cce_from_samples(ys, sws):
    return sum([cce_object(ys, s) * w for s, w, in sws])

def nll_from_samples(ys, sws, regularizer=None):
    if regularizer is None:
        regularizer = 0
    
    mean, variance = tf.nn.weighted_moments(
        tf.stack([s for s, _ in sws]),
        [0],
        tf.stack([w for _, w in sws]))
    if len(sws) > 1 and tf.math.reduce_mean(variance) > regularizer:
        nll = - tf.reduce_mean(tfd.Normal(mean, tf.math.sqrt(variance)).log_prob(ys))
    else:
        nll = None
    
    return nll

Load models:

In [ ]:
dnn = create_dnn()
dnn.load_weights("%s-%s" % (dnn_path, OCC_FLAG))

bnn = create_bnn()
bnn.load_weights("%s-%s" % (bnn_path, OCC_FLAG))

Test DNN:

In [ ]:
batch = 1

sampling = lambda model, xs: [(model(xs), 1.0)]
msec_ftn = lambda model, xs, ys: msec_from_samples(ys, OCC_CLASS_NO, sampling(model, xs))
cce_ftn = lambda model, xs, ys: cce_from_samples(ys, sampling(model, xs))

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

test(dnn, x_test, y_test, batch, runtime_metric, (msec_ftn, mse_metric), (cce_ftn, cce_metric))
        
template = 'DNN Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

Test MU:

In [ ]:
batch, sample_no = 1, 5

sampling = lambda model, xs: [(tf.one_hot(bnn_categorical(model, xs).sample(), OCC_CLASS_NO), 1 / sample_no) for _ in range(sample_no)]
msec_ftn = lambda model, xs, ys: msec_from_samples(ys, OCC_CLASS_NO, sampling(model, xs))
cce_ftn = lambda model, xs, ys: cce_from_samples(ys, sampling(model, xs))

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

test(bnn, x_test, y_test, batch, runtime_metric, (msec_ftn, mse_metric), (cce_ftn, cce_metric))

template = 'MU Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

Test DU:

In [ ]:
batch = 1

x_dims, y_dims = [OCC_INPUT_DIM], [OCC_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
du = DBNN(lambda x: dnn(tf.stack([x[0]])), och_x_1, och_x, och_y)

def sampling(model, xs):
    model.update(xs)
    return model.och_y.cws()

msec_ftn = lambda model, xs, ys: msec_from_samples(ys, OCC_CLASS_NO, sampling(model, xs))
cce_ftn = lambda model, xs, ys: cce_from_samples(ys, sampling(model, xs))

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')


test(du, x_test, y_test, batch, runtime_metric, (msec_ftn, mse_metric), (cce_ftn, cce_metric))

template = 'DU Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

Test DBNN:

In [ ]:
batch = 1

x_dims, y_dims = [OCC_CLASS_NO], [OCC_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
dbnn = DBNN(lambda x: bnn_categorical(bnn, x[0]).sample(), och_x_1, och_x, och_y)

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
cce_metric = tf.keras.metrics.Mean(name='cce')

def sampling(model, xs):
    model.update(xs)
    return model.och_y.cws()

msec_ftn = lambda model, xs, ys: msec_from_samples(ys, OCC_CLASS_NO, sampling(model, xs))
cce_ftn = lambda model, xs, ys: cce_from_samples(ys, sampling(model, xs))

test(du, x_test, y_test, batch, runtime_metric, (msec_ftn, mse_metric), (cce_ftn, cce_metric))

template = 'DBNN Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      cce_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
cce_metric.reset_states()

## [Air Quality Data Set](https://archive.ics.uci.edu/ml/datasets/Air+quality) (Regression)

In [42]:
AIR_FLAG = "air"
AIR_INPUT_DIM = 12
AIR_CLASS_NO = 1
AIR_REGULARIZER = 0.001

In [7]:
def air_data():
    air_path = dataset_root + "%s/" % AIR_FLAG
    x, y = [], []
    count = 0
    file_name = "AirQualityUCI.csv"
    with open(air_path + file_name, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=';')
        header = next(file_reader)
        for row in file_reader:
            try:
                if "-200" in row:
                    raise ValueError()
                x.append(tf.constant([float(v.replace(',', '.')) for v in row[3: 15]]))
                y.append(tf.constant([float(row[2].replace(',', '.'))]))
            except:
                count += 1 
    print("%d rows are ignored." % count)
    return x, y

xs, ys = air_data()
x_train, y_train = xs[:int(len(xs) * 0.9)], ys[:int(len(ys) * 0.9)]
x_test, y_test = xs[int(len(xs) * 0.9):], ys[int(len(ys) * 0.9):]

print("train dataset: %d, test datasets: %d" % (len(x_train), len(x_test)))
print("x sample: ", x_train[0].numpy())
print("y sample: ", y_train[0].numpy())

8644 rows are ignored.
train dataset: 744, test datasets: 83
x sample:  [1.360e+03 1.500e+02 1.190e+01 1.046e+03 1.660e+02 1.056e+03 1.130e+02
 1.692e+03 1.268e+03 1.360e+01 4.890e+01 7.578e-01]
y sample:  [2.6]


In [8]:
def create_dnn():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(50, activation=tf.nn.relu),
        tf.keras.layers.Dense(50, activation=tf.nn.relu),
        tf.keras.layers.Dense(AIR_CLASS_NO)
    ])

def create_bnn():
    return tf.keras.Sequential([
        tfp.layers.DenseFlipout(50, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(50, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(AIR_CLASS_NO * 2)
    ])

### Train Models

Train DNN:

In [ ]:
epochs, batch = 200, 10

dnn = create_dnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_obj = tf.keras.losses.MSE
loss_ftn = lambda model, xs, ys: dnn_loss_ftn(model, xs, ys, loss_obj)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for epoch in range(epochs):
    train_epoch(dnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss)
    
    if (epoch + 1) % 20 == 0:
        template = 'Epoch {}, Loss: {}'
        print(template.format(epoch + 1,
                              train_loss.result()))
        train_loss.reset_states()
        
dnn.save_weights("%s-%s" % (dnn_path, AIR_FLAG), save_format='tf')

Train BNN:

In [ ]:
epochs, batch = 100, 10

bnn = create_bnn()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_ftn = lambda model, xs, ys: bnn_normal_loss_ftn(model, xs, ys, len(x_train), regularizer=AIR_REGULARIZER)
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
    
    if (epoch + 1) % 10 == 0:
        template = 'Epoch {}, Loss: {}, NLL: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (bnn_path, AIR_FLAG), save_format='tf')

### Test Model

In [10]:
dnn = create_dnn()
dnn.load_weights("%s-%s" % (dnn_path, AIR_FLAG))

bnn = create_bnn()
bnn.load_weights("%s-%s" % (bnn_path, AIR_FLAG))

Test DNN:

In [12]:
batch = 1

sampling = lambda model, xs: [(model(xs), 1.0)]
mse_ftn = lambda model, xs, ys: mse_from_samples(ys, sampling(model, xs))

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')

test(dnn, x_test, y_test, batch, runtime_metric, (mse_ftn, mse_metric))
        
template = 'DNN Runtime: {} (ms), MSE: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()

DNN Runtime: 7.243193626403809 (ms), MSE: 0.3162651062011719


Test MU:

In [28]:
batch, sample_no = 1, 5

sampling = lambda model, xs: [(bnn_normal(model, xs, AIR_REGULARIZER).sample(), 1 / sample_no) for _ in range(sample_no)]
mse_ftn = lambda model, xs, ys: mse_from_samples(ys, sampling(model, xs))
nll_ftn = lambda model, xs, ys: nll_from_samples(ys, sampling(model, xs))

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
nll_metric = tf.keras.metrics.Mean(name='nll')

test(bnn, x_test, y_test, batch, runtime_metric, (mse_ftn, mse_metric), (nll_ftn, nll_metric))

template = 'MU Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      nll_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
nll_metric.reset_states()

MU Runtime: 255.9471435546875 (ms), MSE: 92826.71875, NLL: 6.56217622756958


Test DU:

In [52]:
batch = 1

x_dims, y_dims = [AIR_INPUT_DIM], [AIR_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
du = DBNN(lambda x: dnn(tf.stack([x[0]])), och_x_1, och_x, och_y)

def sampling(model, xs):
    model.update(xs)
    return model.och_y.cws()

mse_ftn = lambda model, xs, ys: mse_from_samples(ys, sampling(model, xs))

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
nll_metric = tf.keras.metrics.Mean(name='nll')

test(du, x_test, y_test, batch, runtime_metric, (mse_ftn, mse_metric))

template = 'DU Runtime: {} (ms), MSE: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()

IndexError: tuple index out of range

Test DBNN:

In [51]:
batch = 1

x_dims, y_dims = [AIR_INPUT_DIM], [AIR_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
dbnn = DBNN(lambda x: bnn_normal(bnn, x[0], AIR_REGULARIZER).sample(), och_x_1, och_x, och_y)

def sampling(model, xs):
    model.update(xs)
    return [(c[0], w) for c, w in model.och_y.cws()]

mse_ftn = lambda model, xs, ys: mse_from_samples(ys, sampling(model, xs))
nll_ftn = lambda model, xs, ys: nll_from_samples(ys, sampling(model, xs), AIR_REGULARIZER)

runtime_metric = tf.keras.metrics.Mean(name='runtime')
mse_metric = tf.keras.metrics.Mean(name='mse')
nll_metric = tf.keras.metrics.Mean(name='nll')

test(du, x_test, y_test, batch, runtime_metric, (mse_ftn, mse_metric), (nll_ftn, nll_metric))

template = 'DBNN Runtime: {} (ms), MSE: {}, NLL: {}'
print(template.format(runtime_metric.result() * 1000, 
                      mse_metric.result(),
                      nll_metric.result()))
runtime_metric.reset_states()
mse_metric.reset_states()
nll_metric.reset_states()

DBNN Runtime: 22.532424926757812 (ms), MSE: 1.326663613319397, NLL: 2.824204683303833
